In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%%capture --no-stderr
!pip install python-dotenv langchain-core langchain langchain-openai langchain-chroma

In [ ]:
# 환경변수 설정

In [ ]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [ ]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

# **get_format_instructions 메서드**

In [ ]:
# 라이브러리 설치
!pip install langchain-core langchain langchain-openai langchain-chroma
# 라이브러리 불러오기
from langchain_core.output_parsers import JsonOutputParser
# Json 출력 파서 불러오기
parser = JsonOutputParser()
instructions = parser.get_format_instructions()

# JSON 형식의 지침 출력
print(instructions)

# **parse 메서드**

In [ ]:
ai_response = '{"이름": "김철수", "나이": 30}'
parsed_response = parser.parse(ai_response)

print(parsed_response)

# **parse_with_prompt 메서드**

In [ ]:
# 라이브러리 불러오기
from langchain.output_parsers import RetryWithErrorOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

# 파서 설정
parser = RetryWithErrorOutputParser.from_llm(parser=JsonOutputParser(), llm=ChatOpenAI())

In [ ]:
question = "가장 큰 대륙은?"
# JSON 형식이 아닌 잘못된 응답
ai_response = "아시아입니다."

try:
  result = parser.parse_with_prompt(ai_response, question)
  print(result)
except Exception as e:
  print(f"오류 발생: {e}")
  # 여기서 AI에게 다시 질문할 수 있다

# **PydanticOutputParser**

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_opneai import ChatOpenAI
from pydantic import BaseModel, Field, model_validator

In [ ]:
# OpenAI 모델 설정
model = ChatOpenAI(model_name="gpt-4o", temperature=0.0)

In [ ]:
# 원하는 데이터 구조 정의
class FinancialAdvice(BaseModel):
  setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
  advice: str = Field(description="질문을 해결하기 위한 금융 답변")

  # Pydantic을 사용한 사용자 정의 검증 로직
  @model_validator(mode="before")
  @classmethod
  def question_ends_with_question_mark(cls, values: dict) -> dict:
    setup = values.get("setup", "")

    if not setup.endswith("?"):
      raise ValueError("잘못된 질문 형식입니다! 질문은 '?'로 끝나야 합니다.")

    return values

In [ ]:
# 파서 설정 및 프롬프트 템플릿에 지침 삽입
parser = PydanticOutputParser(pydantic_object=FinancialAdvice)
prompt = PromptTemplate(
    template="""다음 금융 관련 질문에 답변해 주세요.
    {format_instructions}
    질문: {query}""",
    input_variables=["query"],
    # parser.get_format_instructions() 로 생성된 프롬프트가 format_instructions 여기 들어감
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# parser.get_format_instructions()
# LangChain이 Pydantic 스키마를 JSON으로 설명하고,
# “반드시 이 키들만, 큰따옴표, 유효한 JSON, 필요시 코드블록 내 JSON” 같은 엄격한 지침을 생성한다
# 이 지침이 프롬프트에 삽입되어 모델이 형식을 지키도록 유도한다

# parser.get_format_instructions()는 **LangChain의 PydanticOutputParser**가 기본 제공하는 메서드
# Pydantic 모델 기반: 모델 필드/타입/description 정보를 바탕으로 지침 생성
# LLM 출력 안정화: JSON/형식 오류를 줄이는 역할

# model
# 위 지침과 질문을 보고 {"setup": "...?", "advice": "..."} 같은 JSON을 출력한다

# parser
# JSON 파싱 → 2) FinancialAdvice로 로드 → 3) 검증기 실행
# 여기서 @model_validator(mode="before")가 가장 먼저 실행되어 setup이 '?'로 끝나는지 확인한다
# 실패 시 ValueError가 올라오고, LangChain 레이어에서는 보통 파싱 에러로 처리된다

# 언어 모델을 사용해 데이터 주로를 채우도록 프롬프트와 모델 설정
chain = prompt | model | parser

In [ ]:
# 체인 실행 및 결과 출력
try:
  #  template="""다음 금융 관련 질문에 답변해 주세요.
    # JSON 형식과 설정한 Field 지침 삽입
    # 질문: 부동산에 관련하여 금융 조언을 받을 수 있게 질문하라.""",
  result = chain.invoke({"query": "부동산에 관련하여 금융 조언을 받을 수 있게 질문하라."})

  # 답변을 이렇게 지정하였으므로
  # setup: str = Field(description="금융 조언 상황을 설정하기 위한 질문")
  # advice: str = Field(description="질문을 해결하기 위한 금융 답변")

  # setup=질문 상황 설정
  # advice=답변
  # 형식으로 옴
  print(result)
except Exception as e:
  print(f"오류 발생: {e}")

# **SimpleJsonOutputParser**

In [ ]:
from langchain.output_parsers.json import SimpleJsonOutputParser

# JSON 포맷의 응답을 생성하는 프롬프트 템플릿 설정
json_prompt = PromptTemplate.from_template(
    '다음 질문에 대한 답변이 포함된 JSON 객체를 반환하십시오: {question}'
)

json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | model | json_parser

# 스트리밍 예시: 질문에 대합 답변이 부분적으로 구분 분석됨
list(json_chain.stream({"question": "비트코인에 대한 짧은 한문장 설명."}))


# [{},
#  {'description': ''},
#  {'description': '비'},
#  {'description': '비트'},
#  {'description': '비트코'},
# ...
#  {'description': '비트코인은 분산형 디지털 통화로, 중앙은행 없이 개인 간의 거래를 가능하게 하는 블록체인 기술을 기반으로 합니다.'}]